**# HUMAN SCREAM DETECTION**

In [1]:
# Importing the libraries
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import to_categorical
import sounddevice as sd
from scipy.io.wavfile import write
from IPython.display import display, Audio

In [2]:
# Function to extract features from audio files using librosa
def extract_features(file_path, mfcc=True, chroma=True, mel=True):
    y, sr = librosa.load(file_path, mono=True)
    features = []
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
        features.extend(mfccs)
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
        features.extend(chroma)
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr), axis=1)
        features.extend(mel)
    return features


In [3]:
# Example to train the model, evaluate, and test with recorded audio
scream_folder = "Assets/positive"
non_scream_folder = "Assets/negative"

In [4]:
# Function to load and preprocess the dataset
def load_and_preprocess_data(scream_folder, non_scream_folder):
    features = []
    labels = []

    # Load scream data
    import os
    for filename in os.listdir(scream_folder):
        path = os.path.join(scream_folder, filename)
        feature = extract_features(path)
        features.append(feature)
        labels.append(1)  # Use integer labels, e.g., 1 for scream

    # Load non-scream data
    for filename in os.listdir(non_scream_folder):
        path = os.path.join(non_scream_folder, filename)
        feature = extract_features(path)
        features.append(feature)
        labels.append(0)  # Use integer labels, e.g., 0 for non-scream

    X = np.array(features)
    y = np.array(labels)

    # Encode labels using LabelEncoder
    le = LabelEncoder()
    y = le.fit_transform(y)

    # Convert to one-hot encoding
    y = to_categorical(y)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test, le

In [5]:
# Define the model
def create_model(input_shape):
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(input_shape,)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2, activation='softmax'))  # 2 output neurons for binary classification (scream or non-scream)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [8]:
# Evaluate the model
def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Test loss: {loss:.4f}')
    print(f'Test accuracy: {accuracy:.4f}')

In [9]:
# Training the model
def train_model(X_train, y_train, X_test, y_test, epochs=150, batch_size=32):
    input_shape = X_train.shape[1]
    model = create_model(input_shape)

    # Callbacks for early stopping and model checkpoint
    # early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_test, y_test))
    # history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
    #                     validation_data=(X_test, y_test), callbacks=[early_stopping])

    return model, history
X_train, X_test, y_train, y_test, label_encoder = load_and_preprocess_data(scream_folder, non_scream_folder)
model, history = train_model(X_train, y_train, X_test, y_test)
evaluate_model(model, X_test, y_test)

Epoch 1/150
4/4 [==============================] - 1s 95ms/step - loss: 27.6513 - accuracy: 0.5357 - val_loss: 2.1598 - val_accuracy: 0.6897
Epoch 2/150
4/4 [==============================] - 0s 21ms/step - loss: 19.3059 - accuracy: 0.5089 - val_loss: 2.3222 - val_accuracy: 0.7586
Epoch 3/150
4/4 [==============================] - 0s 18ms/step - loss: 15.0003 - accuracy: 0.5179 - val_loss: 2.3160 - val_accuracy: 0.8276
Epoch 4/150
4/4 [==============================] - 0s 19ms/step - loss: 11.2092 - accuracy: 0.6518 - val_loss: 2.2784 - val_accuracy: 0.8276
Epoch 5/150
4/4 [==============================] - 0s 20ms/step - loss: 8.2969 - accuracy: 0.6696 - val_loss: 2.4467 - val_accuracy: 0.8276
Epoch 6/150
4/4 [==============================] - 0s 19ms/step - loss: 6.7503 - accuracy: 0.7321 - val_loss: 2.4860 - val_accuracy: 0.8276
Epoch 7/150
4/4 [==============================] - 0s 22ms/step - loss: 5.8590 - accuracy: 0.7589 - val_loss: 2.4199 - val_accuracy: 0.8276
Epoch 8/150
4/4 

In [10]:
# Function to predict whether an audio file contains a scream or not
def predict_audio(file_path, model, label_encoder):
    
    feature = extract_features(file_path)
    feature = np.array(feature).reshape(1, -1)
    prediction = model.predict(feature)

    if len(prediction) == 0:
        return "Unknown"

    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_label[0]

In [11]:
# Example to test an audio file

# Test : 1

audio_file_to_test = "Assets/testing/n1.wav"
predicted_label = predict_audio(audio_file_to_test, model, label_encoder)
print(f"The predicted label for {audio_file_to_test} is: {predicted_label}")
print()

if predicted_label == 1:
  print("Nearby Officer is alerted")
else:
  print("No alert required")

1/1 [==============================] - 0s 112ms/step
The predicted label for Assets/testing/n1.wav is: 0

No alert required


In [12]:
# Example to test an audio file

# Test : 2

audio_file_to_test = "Assets/testing/p1.wav"
predicted_label = predict_audio(audio_file_to_test, model, label_encoder)
print(f"The predicted label for {audio_file_to_test} is: {predicted_label}")
print()

if predicted_label == 1:
  print("Nearby Officer is alerted")
else:
  print("No alert required")

1/1 [==============================] - 0s 32ms/step
The predicted label for Assets/testing/p1.wav is: 1

Nearby Officer is alerted


In [13]:
# Function to record audio and save it to testing
def record_audio(output_file, duration=10, sample_rate=44100):
    print(f"Recording... Please speak for {duration} seconds.")

    # Record audio
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype=np.int16)
    sd.wait()

    # Save the recorded audio to a file
    write(output_file, sample_rate, audio_data)

    print(f"Recording saved to {output_file}.")

In [16]:
# Example to record audio and test the prediction
output_file_path = "Assets/testing/recorded_audio.wav"
record_audio(output_file_path)

# Predict the label for the recorded audio
predicted_label = predict_audio(output_file_path, model, label_encoder)
print(f"The predicted label for the recorded audio is: {predicted_label}")

if predicted_label == 1:
    print("Nearby Officer is alerted")
else:
    print("No alert required")

Recording... Please speak for 10 seconds.
Recording saved to Assets/testing/recorded_audio.wav.
1/1 [==============================] - 0s 32ms/step
The predicted label for the recorded audio is: 1
Nearby Officer is alerted
